In [2]:
import pandas as pd
import os

# 파일 경로 설정
base_path = r'C:\Users\henaj\Downloads\파이널 프로젝트\기상데이터'
input_file = os.path.join(base_path, '기상데이터_20150630-20250630_원본.csv')
output_file = os.path.join(base_path, '일별_기상데이터.csv')

# CSV 불러오기
df = pd.read_csv(input_file, encoding='cp949')

# '지역(시)', '지역(군)' 컬럼 삭제 (존재할 때만 삭제)
df = df.drop(columns=['지역(시)', '지역(군)'], errors='ignore')

# '조회일자' 기준으로 그룹화해서 나머지 컬럼 평균 계산
df_daily = df.groupby('조회일자', as_index=False).mean(numeric_only=True)

# 결과 저장
df_daily.to_csv(output_file, index=False, encoding='utf-8-sig')

print("✅ 작업 완료! 결과 파일이 저장되었습니다:", output_file)


✅ 작업 완료! 결과 파일이 저장되었습니다: C:\Users\henaj\Downloads\파이널 프로젝트\기상데이터\일별_기상데이터.csv


In [3]:
# -*- coding: utf-8 -*-
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from scipy import stats
import math

# ---------- 설정 ----------
base_path = r'C:\Users\henaj\Downloads\파이널 프로젝트\기상데이터'
input_file = os.path.join(base_path, '일별_기상데이터.csv')
output_excel = os.path.join(base_path, '일별_기상데이터_EDA.xlsx')
plots_folder = os.path.join(base_path, 'EDA_plots')

os.makedirs(plots_folder, exist_ok=True)

# 분석할 열 목록 (질문에서 주신 열 그대로)
cols = [
    '평균 기온(°C)', '최고 기온(°C)', '최저 기온(°C)', '전년 기온(°C)', '평년 기온(°C)', '평년 최고 기온(°C)', '평년 최저 기온(°C)',
    '평균 강수량(mm)', '전년 강수량(mm)', '평년 강수량(mm)',
    '평균 일조시간(hr)', '전년 일조시간(hr)', '평년 일조시간(hr)',
    '평균 일사량(MJ/㎡)', '전년 일사량(MJ/㎡)', '평년 일사량(MJ/㎡)',
    '평균 습도(%)', '전년 습도(%)', '평년 습도(%)',
    '평균 운량(1/10)', '전년 운량(1/10)', '평년 운량(1/10)',
    '평균 적설량(cm)', '전년 적설량(cm)', '평년 적설량(cm)',
    '평균 순간최대풍속(m/s)', '전년 순간최대풍속(m/s)', '평년 순간최대풍속(m/s)'
]

# ---------- 데이터 불러오기 ----------
# 인코딩 문제 발생 시 encoding='cp949'로 변경하세요.
df = pd.read_csv(input_file, encoding='utf-8-sig')

# '조회일자' 기준 정렬(필요시)
if '조회일자' in df.columns:
    # 조회일자 숫자형(예: 20250630) -> 문자열로 변환 후 날짜 타입으로 바꾸면 편함
    try:
        df['조회일자_dt'] = pd.to_datetime(df['조회일자'].astype(str), format='%Y%m%d')
        df = df.sort_values('조회일자_dt', ascending=False).drop(columns=['조회일자_dt'])
    except Exception:
        # 변환 실패하면 그냥 넘어감
        pass

# 필요한 열만 남기되, 존재하지 않는 열은 제외
available_cols = [c for c in cols if c in df.columns]
if len(available_cols) < 1:
    raise ValueError("분석할 열이 데이터에 없습니다. 파일의 열 이름을 확인해 주세요.")

df_sub = df[ ['조회일자'] + available_cols ].copy()

# 숫자형으로 변환 (문자열 중 쉼표, 공백 등 제거)
for c in available_cols:
    # 쉼표 제거, 공백 제거 후 numeric 변환
    df_sub[c] = pd.to_numeric(df_sub[c].astype(str).str.replace(',', '').str.strip(), errors='coerce')

# ---------- 요약 통계 ----------
summary = df_sub[available_cols].agg(['count','mean','median','min','max','std']).transpose()
summary = summary.rename(columns={'count':'N','median':'median','min':'min','max':'max','std':'std'})

# 결측치 정보
summary['missing_count'] = df_sub.shape[0] - summary['N']

# ---------- 정규성 검사 ----------
# 샘플 수가 5000 이하이면 Shapiro 권장, 그보다 크면 normaltest(D'Agostino) 사용
normality_results = []
n_rows = df_sub.shape[0]
for c in available_cols:
    series = df_sub[c].dropna()
    n = len(series)
    if n < 3:
        normality_results.append({'variable':c, 'test':'none', 'stat':np.nan, 'pvalue':np.nan, 'note':'not enough data'})
        continue
    try:
        if n <= 5000:
            stat, p = stats.shapiro(series)
            normality_results.append({'variable':c, 'test':'shapiro', 'stat':stat, 'pvalue':p,
                                      'note':('normal' if p>0.05 else 'not_normal')})
        else:
            # normaltest requires n>=8
            stat, p = stats.normaltest(series)
            normality_results.append({'variable':c, 'test':'normaltest', 'stat':stat, 'pvalue':p,
                                      'note':('normal' if p>0.05 else 'not_normal')})
    except Exception as e:
        normality_results.append({'variable':c, 'test':'error', 'stat':np.nan, 'pvalue':np.nan, 'note':str(e)})

normality_df = pd.DataFrame(normality_results).set_index('variable')

# ---------- 이상치 탐지 (IQR 방법) ----------
outlier_info = []
iqr_details = {}
for c in available_cols:
    s = df_sub[c].dropna()
    if len(s) == 0:
        outlier_info.append({'variable':c, 'n_outliers':0, 'pct_outliers':0.0, 'lower_bound':np.nan, 'upper_bound':np.nan})
        iqr_details[c] = {'Q1':np.nan,'Q3':np.nan,'IQR':np.nan}
        continue
    Q1 = s.quantile(0.25)
    Q3 = s.quantile(0.75)
    IQR = Q3 - Q1
    lower = Q1 - 1.5 * IQR
    upper = Q3 + 1.5 * IQR
    outliers = s[(s < lower) | (s > upper)]
    n_out = outliers.shape[0]
    pct = n_out / df_sub.shape[0] * 100
    outlier_info.append({'variable':c, 'n_outliers':n_out, 'pct_outliers':pct, 'lower_bound':lower, 'upper_bound':upper})
    iqr_details[c] = {'Q1':Q1,'Q3':Q3,'IQR':IQR}

outliers_df = pd.DataFrame(outlier_info).set_index('variable')

# ---------- 상관관계 (Pearson) 및 p-values ----------
vars_for_corr = available_cols.copy()
corr_matrix = df_sub[vars_for_corr].corr(method='pearson')

# p-value matrix
pval_matrix = pd.DataFrame(index=vars_for_corr, columns=vars_for_corr, dtype=float)
for i in range(len(vars_for_corr)):
    for j in range(len(vars_for_corr)):
        x = df_sub[vars_for_corr[i]].dropna()
        y = df_sub[vars_for_corr[j]].dropna()
        # To compute pearsonr, need aligned data: drop NaN pairs
        common = df_sub[[vars_for_corr[i], vars_for_corr[j]]].dropna()
        if common.shape[0] < 3:
            pval_matrix.iloc[i,j] = np.nan
        else:
            try:
                r, p = stats.pearsonr(common[vars_for_corr[i]], common[vars_for_corr[j]])
                pval_matrix.iloc[i,j] = p
            except Exception:
                pval_matrix.iloc[i,j] = np.nan

# ---------- 시각화 생성 ----------
# 1) 각 변수별 히스토그램 + 밀도
hist_files = []
for c in available_cols:
    fig, ax = plt.subplots(figsize=(6,4))
    series = df_sub[c].dropna()
    if len(series) > 0:
        ax.hist(series, bins=30, density=True, alpha=0.6)
        try:
            density = stats.gaussian_kde(series)
            xs = np.linspace(series.min(), series.max(), 200)
            ax.plot(xs, density(xs))
        except Exception:
            pass
    ax.set_title(f'Histogram & Density: {c}')
    ax.set_xlabel(c)
    ax.set_ylabel('Density')
    fname = os.path.join(plots_folder, f'hist_{c.replace("/", "_").replace(" ", "_")}.png')
    fig.tight_layout()
    fig.savefig(fname, dpi=150)
    plt.close(fig)
    hist_files.append(fname)


# 2) 박스플롯: 여러 변수 한 장에 (그리드)
n_vars = len(available_cols)
cols_grid = 4
rows_grid = math.ceil(n_vars / cols_grid)
fig, axes = plt.subplots(rows_grid, cols_grid, figsize=(cols_grid*4, rows_grid*3))
axes = axes.flatten()
for i, c in enumerate(available_cols):
    ax = axes[i]
    series = df_sub[c].dropna()
    if len(series) > 0:
        ax.boxplot(series, vert=True, patch_artist=False)
    ax.set_title(c)
    ax.tick_params(axis='x', which='both', bottom=False, top=False, labelbottom=False)
# 빈 축 숨기기
for k in range(n_vars, len(axes)):
    axes[k].axis('off')
fig.tight_layout()
boxplot_file = os.path.join(plots_folder, 'boxplots_all_vars.png')
fig.savefig(boxplot_file, dpi=150)
plt.close(fig)

# 3) 상관행렬 히트맵 (matplotlib만 사용)
# 간단한 heatmap을 직접 그림
fig, ax = plt.subplots(figsize=(max(8, n_vars*0.25), max(6, n_vars*0.25)))
cax = ax.imshow(corr_matrix.values, interpolation='nearest', vmin=-1, vmax=1)
ax.set_xticks(np.arange(len(vars_for_corr)))
ax.set_yticks(np.arange(len(vars_for_corr)))
ax.set_xticklabels(vars_for_corr, rotation=90, fontsize=8)
ax.set_yticklabels(vars_for_corr, fontsize=8)
fig.colorbar(cax, ax=ax, fraction=0.03)
ax.set_title('Pearson Correlation Matrix')
fig.tight_layout()
corr_file = os.path.join(plots_folder, 'correlation_matrix.png')
fig.savefig(corr_file, dpi=150)
plt.close(fig)

# ---------- 엑셀로 저장 (숫자 결과 + 이미지 삽입) ----------
with pd.ExcelWriter(output_excel, engine='xlsxwriter') as writer:
    # 요약 통계
    summary.to_excel(writer, sheet_name='summary_stats')
    # 정규성
    normality_df.to_excel(writer, sheet_name='normality_tests')
    # 이상치
    outliers_df.to_excel(writer, sheet_name='outlier_summary')
    # 상관계수 및 p-value
    corr_matrix.to_excel(writer, sheet_name='correlation_matrix')
    pval_matrix.to_excel(writer, sheet_name='correlation_pvalues')
    # 원본(선택한 열)도 저장
    df_sub.to_excel(writer, sheet_name='data_sample', index=False)

    # 이미지 삽입: 새로운 시트 생성
    workbook  = writer.book
    try:
        worksheet = workbook.add_worksheet('Plots')
    except Exception:
        worksheet = writer.sheets.get('Plots') or workbook.add_worksheet('Plots')

    # 이미지 위치 지정 (행, 열)
    row = 0
    col = 0
    # 먼저 히스토그램들 (한 행에 2개씩)
    for i, fpath in enumerate(hist_files):
        # Excel cell 위치 (row, col) in pixels offset handled by xlsxwriter
        worksheet.insert_image(row, col, fpath, {'x_scale':0.9, 'y_scale':0.9})
        col += 8  # 다음 이미지 위한 열 이동
        if (i+1) % 2 == 0:
            row += 20
            col = 0
    # 박스플롯
    row += 22
    col = 0
    worksheet.insert_image(row, col, boxplot_file, {'x_scale':0.9, 'y_scale':0.9})
    row += 30
    # 상관행렬 이미지
    worksheet.insert_image(row, 0, corr_file, {'x_scale':0.9, 'y_scale':0.9})

# ---------- 콘솔 출력 요약 ----------
print("EDA 완료.")
print(f"결과 엑셀 파일: {output_excel}")
print(f"플롯 이미지 폴더: {plots_folder}")
print("\n요약 통계 (상위 10개 변수):")
print(summary.head(10))
print("\n정규성 테스트 결과 (샘플):")
print(normality_df.head(10))
print("\n이상치 요약 (상위 10개 변수):")
print(outliers_df.sort_values('n_outliers', ascending=False).head(10))
print("\n상관계수(일부):")
print(corr_matrix.iloc[:5,:5])

# 끝


C:\Users\henaj\anaconda3\Lib\site-packages\scipy\stats\_axis_nan_policy.py:531: UserWarning: scipy.stats.shapiro: Input data has range zero. The results may not be accurate.
  res = hypotest_fun_out(*samples, **kwds)
C:\Users\henaj\AppData\Local\Temp\ipykernel_13684\93539023.py:125: ConstantInputWarning: An input array is constant; the correlation coefficient is not defined.
  r, p = stats.pearsonr(common[vars_for_corr[i]], common[vars_for_corr[j]])
C:\Users\henaj\AppData\Local\Temp\ipykernel_13684\93539023.py:125: ConstantInputWarning: An input array is constant; the correlation coefficient is not defined.
  r, p = stats.pearsonr(common[vars_for_corr[i]], common[vars_for_corr[j]])
C:\Users\henaj\AppData\Local\Temp\ipykernel_13684\93539023.py:125: ConstantInputWarning: An input array is constant; the correlation coefficient is not defined.
  r, p = stats.pearsonr(common[vars_for_corr[i]], common[vars_for_corr[j]])
C:\Users\henaj\AppData\Local\Temp\ipykernel_13684\93539023.py:125: Const

EDA 완료.
결과 엑셀 파일: C:\Users\henaj\Downloads\파이널 프로젝트\기상데이터\일별_기상데이터_EDA.xlsx
플롯 이미지 폴더: C:\Users\henaj\Downloads\파이널 프로젝트\기상데이터\EDA_plots

요약 통계 (상위 10개 변수):
                   N       mean     median        min         max       std  \
평균 기온(°C)     3652.0  13.398069  14.111268 -99.000000   29.888235  9.765127   
최고 기온(°C)     3652.0  18.845144  20.236050  -7.229412   36.364706  9.369548   
최저 기온(°C)     3652.0   8.847502   8.954478 -15.794030   26.304225  9.875944   
전년 기온(°C)     3652.0  13.257304  13.931102 -99.000000   29.888235  9.432100   
평년 기온(°C)     3652.0  12.088742  12.817647  -1.268657   25.350746  8.597256   
평년 최고 기온(°C)  3652.0  17.332790  18.764464   3.597059   29.841791  8.374253   
평년 최저 기온(°C)  3652.0   7.576064   7.246479  -5.788060   22.002817  8.977317   
평균 강수량(mm)    3652.0   3.594690   0.205926   0.000000  108.938028  9.043281   
전년 강수량(mm)    3652.0   3.555772   0.194242   0.000000  104.902985  8.857365   
평년 강수량(mm)    3652.0   3.517436   2.281690   0.354412

In [8]:
# 위에서 EDA를 다해 해서 결과물을 엑셀파일과 폴더로 다 EXPORT해서 저장했지만, BOX PLOT들이 깨져 나왔으므로
# 깨지지 않게 박스플롯 그리는 부분만 코딩을 좀더 추가해서 처음부터 다시 EDA해 엑셀파일과 폴더 같은 곳으로 다시 EXPORT합니다.

# -*- coding: utf-8 -*-
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from scipy import stats
import math

# ---------- 설정 ----------
base_path = r'C:\Users\henaj\Downloads\파이널 프로젝트\기상데이터'
input_file = os.path.join(base_path, '일별_기상데이터.csv')
output_excel = os.path.join(base_path, '일별_기상데이터_EDA.xlsx')
plots_folder = os.path.join(base_path, 'EDA_plots')

os.makedirs(plots_folder, exist_ok=True)

# 분석할 열 목록 (질문에서 주신 열 그대로)
cols = [
    '평균 기온(°C)', '최고 기온(°C)', '최저 기온(°C)', '전년 기온(°C)', '평년 기온(°C)', '평년 최고 기온(°C)', '평년 최저 기온(°C)',
    '평균 강수량(mm)', '전년 강수량(mm)', '평년 강수량(mm)',
    '평균 일조시간(hr)', '전년 일조시간(hr)', '평년 일조시간(hr)',
    '평균 일사량(MJ/㎡)', '전년 일사량(MJ/㎡)', '평년 일사량(MJ/㎡)',
    '평균 습도(%)', '전년 습도(%)', '평년 습도(%)',
    '평균 운량(1/10)', '전년 운량(1/10)', '평년 운량(1/10)',
    '평균 적설량(cm)', '전년 적설량(cm)', '평년 적설량(cm)',
    '평균 순간최대풍속(m/s)', '전년 순간최대풍속(m/s)', '평년 순간최대풍속(m/s)'
]

# ---------- 데이터 불러오기 ----------
# 인코딩 문제 발생 시 encoding='cp949'로 변경하세요.
df = pd.read_csv(input_file, encoding='utf-8-sig')

# '조회일자' 기준 정렬(필요시)
if '조회일자' in df.columns:
    # 조회일자 숫자형(예: 20250630) -> 문자열로 변환 후 날짜 타입으로 바꾸면 편함
    try:
        df['조회일자_dt'] = pd.to_datetime(df['조회일자'].astype(str), format='%Y%m%d')
        df = df.sort_values('조회일자_dt', ascending=False).drop(columns=['조회일자_dt'])
    except Exception:
        # 변환 실패하면 그냥 넘어감
        pass

# 필요한 열만 남기되, 존재하지 않는 열은 제외
available_cols = [c for c in cols if c in df.columns]
if len(available_cols) < 1:
    raise ValueError("분석할 열이 데이터에 없습니다. 파일의 열 이름을 확인해 주세요.")

df_sub = df[ ['조회일자'] + available_cols ].copy()

# 숫자형으로 변환 (문자열 중 쉼표, 공백 등 제거)
for c in available_cols:
    # 쉼표 제거, 공백 제거 후 numeric 변환
    df_sub[c] = pd.to_numeric(df_sub[c].astype(str).str.replace(',', '').str.strip(), errors='coerce')

# ---------- 요약 통계 ----------
summary = df_sub[available_cols].agg(['count','mean','median','min','max','std']).transpose()
summary = summary.rename(columns={'count':'N','median':'median','min':'min','max':'max','std':'std'})

# 결측치 정보
summary['missing_count'] = df_sub.shape[0] - summary['N']

# ---------- 정규성 검사 ----------
# 샘플 수가 5000 이하이면 Shapiro 권장, 그보다 크면 normaltest(D'Agostino) 사용
normality_results = []
n_rows = df_sub.shape[0]
for c in available_cols:
    series = df_sub[c].dropna()
    n = len(series)
    if n < 3:
        normality_results.append({'variable':c, 'test':'none', 'stat':np.nan, 'pvalue':np.nan, 'note':'not enough data'})
        continue
    try:
        if n <= 5000:
            stat, p = stats.shapiro(series)
            normality_results.append({'variable':c, 'test':'shapiro', 'stat':stat, 'pvalue':p,
                                      'note':('normal' if p>0.05 else 'not_normal')})
        else:
            # normaltest requires n>=8
            stat, p = stats.normaltest(series)
            normality_results.append({'variable':c, 'test':'normaltest', 'stat':stat, 'pvalue':p,
                                      'note':('normal' if p>0.05 else 'not_normal')})
    except Exception as e:
        normality_results.append({'variable':c, 'test':'error', 'stat':np.nan, 'pvalue':np.nan, 'note':str(e)})

normality_df = pd.DataFrame(normality_results).set_index('variable')

# ---------- 이상치 탐지 (IQR 방법) ----------
outlier_info = []
iqr_details = {}
for c in available_cols:
    s = df_sub[c].dropna()
    if len(s) == 0:
        outlier_info.append({'variable':c, 'n_outliers':0, 'pct_outliers':0.0, 'lower_bound':np.nan, 'upper_bound':np.nan})
        iqr_details[c] = {'Q1':np.nan,'Q3':np.nan,'IQR':np.nan}
        continue
    Q1 = s.quantile(0.25)
    Q3 = s.quantile(0.75)
    IQR = Q3 - Q1
    lower = Q1 - 1.5 * IQR
    upper = Q3 + 1.5 * IQR
    outliers = s[(s < lower) | (s > upper)]
    n_out = outliers.shape[0]
    pct = n_out / df_sub.shape[0] * 100
    outlier_info.append({'variable':c, 'n_outliers':n_out, 'pct_outliers':pct, 'lower_bound':lower, 'upper_bound':upper})
    iqr_details[c] = {'Q1':Q1,'Q3':Q3,'IQR':IQR}

outliers_df = pd.DataFrame(outlier_info).set_index('variable')

# ---------- 상관관계 (Pearson) 및 p-values ----------
vars_for_corr = available_cols.copy()
corr_matrix = df_sub[vars_for_corr].corr(method='pearson')

# p-value matrix
pval_matrix = pd.DataFrame(index=vars_for_corr, columns=vars_for_corr, dtype=float)
for i in range(len(vars_for_corr)):
    for j in range(len(vars_for_corr)):
        x = df_sub[vars_for_corr[i]].dropna()
        y = df_sub[vars_for_corr[j]].dropna()
        # To compute pearsonr, need aligned data: drop NaN pairs
        common = df_sub[[vars_for_corr[i], vars_for_corr[j]]].dropna()
        if common.shape[0] < 3:
            pval_matrix.iloc[i,j] = np.nan
        else:
            try:
                r, p = stats.pearsonr(common[vars_for_corr[i]], common[vars_for_corr[j]])
                pval_matrix.iloc[i,j] = p
            except Exception:
                pval_matrix.iloc[i,j] = np.nan

# ---------- 시각화 생성 ----------
# 1) 각 변수별 히스토그램 + 밀도 - 한글 폰트 적용
hist_files = []
for c in available_cols:
    fig, ax = plt.subplots(figsize=(8,5))
    series = df_sub[c].dropna()
    if len(series) > 0:
        ax.hist(series, bins=30, density=True, alpha=0.6, color='skyblue', edgecolor='black')
        try:
            density = stats.gaussian_kde(series)
            xs = np.linspace(series.min(), series.max(), 200)
            ax.plot(xs, density(xs), color='red', linewidth=2)
        except Exception:
            pass
    ax.set_title(f'Histogram & Density: {c}', fontsize=12, pad=15)
    ax.set_xlabel(c, fontsize=10)
    ax.set_ylabel('Density', fontsize=10)
    ax.grid(True, alpha=0.3)
    
    fname = os.path.join(plots_folder, f'hist_{c.replace("/", "_").replace(" ", "_").replace("(", "").replace(")", "").replace("°", "deg").replace("㎡", "m2")}.png')
    fig.tight_layout()
    fig.savefig(fname, dpi=150, bbox_inches='tight')
    plt.close(fig)
    hist_files.append(fname)

# 2) 박스플롯: 여러 변수 한 장에 (그리드)
fig, ax = plt.subplots(figsize=(max(8, len(available_cols)*0.25), 
                                max(6, len(available_cols)*0.25)))
df_sub[available_cols].boxplot(ax=ax, rot=90)  # rot=90 → x축 레이블 회전
ax.set_title('Boxplots of Selected Variables')
fig.tight_layout()

boxplot_file = os.path.join(plots_folder, 'boxplots_all_vars.png')
fig.savefig(boxplot_file, dpi=150)
plt.close(fig)

# 3) 상관행렬 히트맵 (matplotlib만 사용)
# 간단한 heatmap을 직접 그림
fig, ax = plt.subplots(figsize=(max(8, len(available_cols)*0.25), max(6, len(available_cols)*0.25)))
cax = ax.imshow(corr_matrix.values, interpolation='nearest', vmin=-1, vmax=1)
ax.set_xticks(np.arange(len(vars_for_corr)))
ax.set_yticks(np.arange(len(vars_for_corr)))
ax.set_xticklabels(vars_for_corr, rotation=90, fontsize=8)
ax.set_yticklabels(vars_for_corr, fontsize=8)
fig.colorbar(cax, ax=ax, fraction=0.03)
ax.set_title('Pearson Correlation Matrix')
fig.tight_layout()
corr_file = os.path.join(plots_folder, 'correlation_matrix.png')
fig.savefig(corr_file, dpi=150)
plt.close(fig)

# ---------- 엑셀로 저장 (숫자 결과 + 이미지 삽입) ----------
with pd.ExcelWriter(output_excel, engine='xlsxwriter') as writer:
    # 요약 통계
    summary.to_excel(writer, sheet_name='summary_stats')
    # 정규성
    normality_df.to_excel(writer, sheet_name='normality_tests')
    # 이상치
    outliers_df.to_excel(writer, sheet_name='outlier_summary')
    # 상관계수 및 p-value
    corr_matrix.to_excel(writer, sheet_name='correlation_matrix')
    pval_matrix.to_excel(writer, sheet_name='correlation_pvalues')
    # 원본(선택한 열)도 저장
    df_sub.to_excel(writer, sheet_name='data_sample', index=False)

    # 이미지 삽입: 새로운 시트 생성
    workbook  = writer.book
    try:
        worksheet = workbook.add_worksheet('Plots')
    except Exception:
        worksheet = writer.sheets.get('Plots') or workbook.add_worksheet('Plots')

    # 이미지 위치 지정 (행, 열)
    row = 0
    col = 0
    # 먼저 히스토그램들 (한 행에 2개씩)
    for i, fpath in enumerate(hist_files):
        # Excel cell 위치 (row, col) in pixels offset handled by xlsxwriter
        worksheet.insert_image(row, col, fpath, {'x_scale':0.9, 'y_scale':0.9})
        col += 8  # 다음 이미지 위한 열 이동
        if (i+1) % 2 == 0:
            row += 20
            col = 0
    # 박스플롯
    row += 22
    col = 0
    worksheet.insert_image(row, col, boxplot_file, {'x_scale':0.9, 'y_scale':0.9})
    row += 30
    # 상관행렬 이미지
    worksheet.insert_image(row, 0, corr_file, {'x_scale':0.9, 'y_scale':0.9})

# ---------- 콘솔 출력 요약 ----------
print("EDA 완료.")
print(f"결과 엑셀 파일: {output_excel}")
print(f"플롯 이미지 폴더: {plots_folder}")
print("\n요약 통계 (모든 변수):")
print(summary)
print("\n정규성 테스트 결과:")
print(normality_df)
print("\n이상치 요약 (모든 변수):")
print(outliers_df.sort_values('n_outliers', ascending=False))
print("\n상관계수:")
print(corr_matrix)

# 끝

C:\Users\henaj\anaconda3\Lib\site-packages\scipy\stats\_axis_nan_policy.py:531: UserWarning: scipy.stats.shapiro: Input data has range zero. The results may not be accurate.
  res = hypotest_fun_out(*samples, **kwds)
C:\Users\henaj\AppData\Local\Temp\ipykernel_37308\3458932083.py:128: ConstantInputWarning: An input array is constant; the correlation coefficient is not defined.
  r, p = stats.pearsonr(common[vars_for_corr[i]], common[vars_for_corr[j]])
C:\Users\henaj\AppData\Local\Temp\ipykernel_37308\3458932083.py:128: ConstantInputWarning: An input array is constant; the correlation coefficient is not defined.
  r, p = stats.pearsonr(common[vars_for_corr[i]], common[vars_for_corr[j]])
C:\Users\henaj\AppData\Local\Temp\ipykernel_37308\3458932083.py:128: ConstantInputWarning: An input array is constant; the correlation coefficient is not defined.
  r, p = stats.pearsonr(common[vars_for_corr[i]], common[vars_for_corr[j]])
C:\Users\henaj\AppData\Local\Temp\ipykernel_37308\3458932083.py:12

EDA 완료.
결과 엑셀 파일: C:\Users\henaj\Downloads\파이널 프로젝트\기상데이터\일별_기상데이터_EDA.xlsx
플롯 이미지 폴더: C:\Users\henaj\Downloads\파이널 프로젝트\기상데이터\EDA_plots

요약 통계 (모든 변수):
                     N       mean     median        min         max  \
평균 기온(°C)       3652.0  13.398069  14.111268 -99.000000   29.888235   
최고 기온(°C)       3652.0  18.845144  20.236050  -7.229412   36.364706   
최저 기온(°C)       3652.0   8.847502   8.954478 -15.794030   26.304225   
전년 기온(°C)       3652.0  13.257304  13.931102 -99.000000   29.888235   
평년 기온(°C)       3652.0  12.088742  12.817647  -1.268657   25.350746   
평년 최고 기온(°C)    3652.0  17.332790  18.764464   3.597059   29.841791   
평년 최저 기온(°C)    3652.0   7.576064   7.246479  -5.788060   22.002817   
평균 강수량(mm)      3652.0   3.594690   0.205926   0.000000  108.938028   
전년 강수량(mm)      3652.0   3.555772   0.194242   0.000000  104.902985   
평년 강수량(mm)      3652.0   3.517436   2.281690   0.354412   13.552239   
평균 일조시간(hr)     3652.0   6.247519   6.858347   0.000000   12.92253

In [1]:
## 깨져 나와서 다시 한 번 더 최종수정


# -*- coding: utf-8 -*-
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from scipy import stats
import math

# ---------- 한글 폰트 설정 추가 ----------
# 이 부분이 가장 중요합니다.
# matplotlib에 시스템의 한글 폰트를 설정하여 글씨 깨짐을 방지합니다.

from matplotlib import font_manager, rc

# 폰트 경로를 찾아서 설정합니다.
# Windows 환경의 경우
font_path = 'C:/Windows/Fonts/malgun.ttf'
# 만약 'malgun.ttf' 폰트가 없다면, 다른 한글 폰트(ex: NanumGothic.ttf) 경로를 지정해주세요.
if os.path.exists(font_path):
    font_name = font_manager.FontProperties(fname=font_path).get_name()
    rc('font', family=font_name)
else:
    print("맑은 고딕 폰트를 찾을 수 없습니다. 다른 한글 폰트를 사용하거나 설치해주세요.")
    # 기본 fallback 폰트로 설정하거나 다른 폰트 경로를 지정
    rc('font', family='AppleGothic') # macOS 환경
    # rc('font', family='NanumGothic') # 리눅스 환경 등

# 마이너스 부호가 깨지는 현상을 방지합니다.
plt.rcParams['axes.unicode_minus'] = False
# ---------------------------------------------


# ---------- 설정 ----------
base_path = r'C:\Users\henaj\Downloads\파이널 프로젝트\기상데이터'
input_file = os.path.join(base_path, '일별_기상데이터.csv')
output_excel = os.path.join(base_path, '일별_기상데이터_EDA.xlsx')
plots_folder = os.path.join(base_path, 'EDA_plots')

os.makedirs(plots_folder, exist_ok=True)

# 분석할 열 목록 (질문에서 주신 열 그대로)
cols = [
    '평균 기온(°C)', '최고 기온(°C)', '최저 기온(°C)', '전년 기온(°C)', '평년 기온(°C)', '평년 최고 기온(°C)', '평년 최저 기온(°C)',
    '평균 강수량(mm)', '전년 강수량(mm)', '평년 강수량(mm)',
    '평균 일조시간(hr)', '전년 일조시간(hr)', '평년 일조시간(hr)',
    '평균 일사량(MJ/㎡)', '전년 일사량(MJ/㎡)', '평년 일사량(MJ/㎡)',
    '평균 습도(%)', '전년 습도(%)', '평년 습도(%)',
    '평균 운량(1/10)', '전년 운량(1/10)', '평년 운량(1/10)',
    '평균 적설량(cm)', '전년 적설량(cm)', '평년 적설량(cm)',
    '평균 순간최대풍속(m/s)', '전년 순간최대풍속(m/s)', '평년 순간최대풍속(m/s)'
]

# ---------- 데이터 불러오기 ----------
# 인코딩 문제 발생 시 encoding='cp949'로 변경하세요.
df = pd.read_csv(input_file, encoding='utf-8-sig')

# '조회일자' 기준 정렬(필요시)
if '조회일자' in df.columns:
    # 조회일자 숫자형(예: 20250630) -> 문자열로 변환 후 날짜 타입으로 바꾸면 편함
    try:
        df['조회일자_dt'] = pd.to_datetime(df['조회일자'].astype(str), format='%Y%m%d')
        df = df.sort_values('조회일자_dt', ascending=False).drop(columns=['조회일자_dt'])
    except Exception:
        # 변환 실패하면 그냥 넘어감
        pass

# 필요한 열만 남기되, 존재하지 않는 열은 제외
available_cols = [c for c in cols if c in df.columns]
if len(available_cols) < 1:
    raise ValueError("분석할 열이 데이터에 없습니다. 파일의 열 이름을 확인해 주세요.")

df_sub = df[ ['조회일자'] + available_cols ].copy()

# 숫자형으로 변환 (문자열 중 쉼표, 공백 등 제거)
for c in available_cols:
    # 쉼표 제거, 공백 제거 후 numeric 변환
    df_sub[c] = pd.to_numeric(df_sub[c].astype(str).str.replace(',', '').str.strip(), errors='coerce')

# ---------- 요약 통계 ----------
summary = df_sub[available_cols].agg(['count','mean','median','min','max','std']).transpose()
summary = summary.rename(columns={'count':'N','median':'median','min':'min','max':'max','std':'std'})

# 결측치 정보
summary['missing_count'] = df_sub.shape[0] - summary['N']

# ---------- 정규성 검사 ----------
# 샘플 수가 5000 이하이면 Shapiro 권장, 그보다 크면 normaltest(D'Agostino) 사용
normality_results = []
n_rows = df_sub.shape[0]
for c in available_cols:
    series = df_sub[c].dropna()
    n = len(series)
    if n < 3:
        normality_results.append({'variable':c, 'test':'none', 'stat':np.nan, 'pvalue':np.nan, 'note':'not enough data'})
        continue
    try:
        if n <= 5000:
            stat, p = stats.shapiro(series)
            normality_results.append({'variable':c, 'test':'shapiro', 'stat':stat, 'pvalue':p,
                                     'note':('normal' if p>0.05 else 'not_normal')})
        else:
            # normaltest requires n>=8
            stat, p = stats.normaltest(series)
            normality_results.append({'variable':c, 'test':'normaltest', 'stat':stat, 'pvalue':p,
                                     'note':('normal' if p>0.05 else 'not_normal')})
    except Exception as e:
        normality_results.append({'variable':c, 'test':'error', 'stat':np.nan, 'pvalue':np.nan, 'note':str(e)})

normality_df = pd.DataFrame(normality_results).set_index('variable')

# ---------- 이상치 탐지 (IQR 방법) ----------
outlier_info = []
iqr_details = {}
for c in available_cols:
    s = df_sub[c].dropna()
    if len(s) == 0:
        outlier_info.append({'variable':c, 'n_outliers':0, 'pct_outliers':0.0, 'lower_bound':np.nan, 'upper_bound':np.nan})
        iqr_details[c] = {'Q1':np.nan,'Q3':np.nan,'IQR':np.nan}
        continue
    Q1 = s.quantile(0.25)
    Q3 = s.quantile(0.75)
    IQR = Q3 - Q1
    lower = Q1 - 1.5 * IQR
    upper = Q3 + 1.5 * IQR
    outliers = s[(s < lower) | (s > upper)]
    n_out = outliers.shape[0]
    pct = n_out / df_sub.shape[0] * 100
    outlier_info.append({'variable':c, 'n_outliers':n_out, 'pct_outliers':pct, 'lower_bound':lower, 'upper_bound':upper})
    iqr_details[c] = {'Q1':Q1,'Q3':Q3,'IQR':IQR}

outliers_df = pd.DataFrame(outlier_info).set_index('variable')

# ---------- 상관관계 (Pearson) 및 p-values ----------
vars_for_corr = available_cols.copy()
corr_matrix = df_sub[vars_for_corr].corr(method='pearson')

# p-value matrix
pval_matrix = pd.DataFrame(index=vars_for_corr, columns=vars_for_corr, dtype=float)
for i in range(len(vars_for_corr)):
    for j in range(len(vars_for_corr)):
        x = df_sub[vars_for_corr[i]].dropna()
        y = df_sub[vars_for_corr[j]].dropna()
        # To compute pearsonr, need aligned data: drop NaN pairs
        common = df_sub[[vars_for_corr[i], vars_for_corr[j]]].dropna()
        if common.shape[0] < 3:
            pval_matrix.iloc[i,j] = np.nan
        else:
            try:
                r, p = stats.pearsonr(common[vars_for_corr[i]], common[vars_for_corr[j]])
                pval_matrix.iloc[i,j] = p
            except Exception:
                pval_matrix.iloc[i,j] = np.nan

# ---------- 시각화 생성 ----------
# 1) 각 변수별 히스토그램 + 밀도 - 한글 폰트 적용
hist_files = []
for c in available_cols:
    fig, ax = plt.subplots(figsize=(8,5))
    series = df_sub[c].dropna()
    if len(series) > 0:
        ax.hist(series, bins=30, density=True, alpha=0.6, color='skyblue', edgecolor='black')
        try:
            density = stats.gaussian_kde(series)
            xs = np.linspace(series.min(), series.max(), 200)
            ax.plot(xs, density(xs), color='red', linewidth=2)
        except Exception:
            pass
    ax.set_title(f'Histogram & Density: {c}', fontsize=12, pad=15)
    ax.set_xlabel(c, fontsize=10)
    ax.set_ylabel('Density', fontsize=10)
    ax.grid(True, alpha=0.3)
    
    fname = os.path.join(plots_folder, f'hist_{c.replace("/", "_").replace(" ", "_").replace("(", "").replace(")", "").replace("°", "deg").replace("㎡", "m2")}.png')
    fig.tight_layout()
    fig.savefig(fname, dpi=150, bbox_inches='tight')
    plt.close(fig)
    hist_files.append(fname)

# 2) 박스플롯: 여러 변수 한 장에 (그리드)
fig, ax = plt.subplots(figsize=(max(8, len(available_cols)*0.25), 
                                max(6, len(available_cols)*0.25)))
df_sub[available_cols].boxplot(ax=ax, rot=90) # rot=90 → x축 레이블 회전
ax.set_title('Boxplots of Selected Variables')
fig.tight_layout()

boxplot_file = os.path.join(plots_folder, 'boxplots_all_vars.png')
fig.savefig(boxplot_file, dpi=150)
plt.close(fig)

# 3) 상관행렬 히트맵 (matplotlib만 사용)
# 간단한 heatmap을 직접 그림
fig, ax = plt.subplots(figsize=(max(8, len(available_cols)*0.25), max(6, len(available_cols)*0.25)))
cax = ax.imshow(corr_matrix.values, interpolation='nearest', vmin=-1, vmax=1)
ax.set_xticks(np.arange(len(vars_for_corr)))
ax.set_yticks(np.arange(len(vars_for_corr)))
ax.set_xticklabels(vars_for_corr, rotation=90, fontsize=8)
ax.set_yticklabels(vars_for_corr, fontsize=8)
fig.colorbar(cax, ax=ax, fraction=0.03)
ax.set_title('Pearson Correlation Matrix')
fig.tight_layout()
corr_file = os.path.join(plots_folder, 'correlation_matrix.png')
fig.savefig(corr_file, dpi=150)
plt.close(fig)

# ---------- 엑셀로 저장 (숫자 결과 + 이미지 삽입) ----------
with pd.ExcelWriter(output_excel, engine='xlsxwriter') as writer:
    # 요약 통계
    summary.to_excel(writer, sheet_name='summary_stats')
    # 정규성
    normality_df.to_excel(writer, sheet_name='normality_tests')
    # 이상치
    outliers_df.to_excel(writer, sheet_name='outlier_summary')
    # 상관계수 및 p-value
    corr_matrix.to_excel(writer, sheet_name='correlation_matrix')
    pval_matrix.to_excel(writer, sheet_name='correlation_pvalues')
    # 원본(선택한 열)도 저장
    df_sub.to_excel(writer, sheet_name='data_sample', index=False)

    # 이미지 삽입: 새로운 시트 생성
    workbook  = writer.book
    try:
        worksheet = workbook.add_worksheet('Plots')
    except Exception:
        worksheet = writer.sheets.get('Plots') or workbook.add_worksheet('Plots')

    # 이미지 위치 지정 (행, 열)
    row = 0
    col = 0
    # 먼저 히스토그램들 (한 행에 2개씩)
    for i, fpath in enumerate(hist_files):
        # Excel cell 위치 (row, col) in pixels offset handled by xlsxwriter
        worksheet.insert_image(row, col, fpath, {'x_scale':0.9, 'y_scale':0.9})
        col += 8  # 다음 이미지 위한 열 이동
        if (i+1) % 2 == 0:
            row += 20
            col = 0
    # 박스플롯
    row += 22
    col = 0
    worksheet.insert_image(row, col, boxplot_file, {'x_scale':0.9, 'y_scale':0.9})
    row += 30
    # 상관행렬 이미지
    worksheet.insert_image(row, 0, corr_file, {'x_scale':0.9, 'y_scale':0.9})

# ---------- 콘솔 출력 요약 ----------
print("EDA 완료.")
print(f"결과 엑셀 파일: {output_excel}")
print(f"플롯 이미지 폴더: {plots_folder}")
print("\n요약 통계 (모든 변수):")
print(summary)
print("\n정규성 테스트 결과:")
print(normality_df)
print("\n이상치 요약 (모든 변수):")
print(outliers_df.sort_values('n_outliers', ascending=False))
print("\n상관계수:")
print(corr_matrix)

# 끝

C:\Users\henaj\anaconda3\Lib\site-packages\scipy\stats\_axis_nan_policy.py:531: UserWarning: scipy.stats.shapiro: Input data has range zero. The results may not be accurate.
  res = hypotest_fun_out(*samples, **kwds)
C:\Users\henaj\AppData\Local\Temp\ipykernel_4168\4037755060.py:152: ConstantInputWarning: An input array is constant; the correlation coefficient is not defined.
  r, p = stats.pearsonr(common[vars_for_corr[i]], common[vars_for_corr[j]])
C:\Users\henaj\AppData\Local\Temp\ipykernel_4168\4037755060.py:152: ConstantInputWarning: An input array is constant; the correlation coefficient is not defined.
  r, p = stats.pearsonr(common[vars_for_corr[i]], common[vars_for_corr[j]])
C:\Users\henaj\AppData\Local\Temp\ipykernel_4168\4037755060.py:152: ConstantInputWarning: An input array is constant; the correlation coefficient is not defined.
  r, p = stats.pearsonr(common[vars_for_corr[i]], common[vars_for_corr[j]])
C:\Users\henaj\AppData\Local\Temp\ipykernel_4168\4037755060.py:152: C

EDA 완료.
결과 엑셀 파일: C:\Users\henaj\Downloads\파이널 프로젝트\기상데이터\일별_기상데이터_EDA.xlsx
플롯 이미지 폴더: C:\Users\henaj\Downloads\파이널 프로젝트\기상데이터\EDA_plots

요약 통계 (모든 변수):
                     N       mean     median        min         max  \
평균 기온(°C)       3652.0  13.398069  14.111268 -99.000000   29.888235   
최고 기온(°C)       3652.0  18.845144  20.236050  -7.229412   36.364706   
최저 기온(°C)       3652.0   8.847502   8.954478 -15.794030   26.304225   
전년 기온(°C)       3652.0  13.257304  13.931102 -99.000000   29.888235   
평년 기온(°C)       3652.0  12.088742  12.817647  -1.268657   25.350746   
평년 최고 기온(°C)    3652.0  17.332790  18.764464   3.597059   29.841791   
평년 최저 기온(°C)    3652.0   7.576064   7.246479  -5.788060   22.002817   
평균 강수량(mm)      3652.0   3.594690   0.205926   0.000000  108.938028   
전년 강수량(mm)      3652.0   3.555772   0.194242   0.000000  104.902985   
평년 강수량(mm)      3652.0   3.517436   2.281690   0.354412   13.552239   
평균 일조시간(hr)     3652.0   6.247519   6.858347   0.000000   12.92253

In [1]:
# -90도씨 이하 데이터 2개를 앞뒤평균보간법으로 대체해서 새로운 파일 '일별_기상데이터_이상치 처리.csv'로 저장
import pandas as pd
import numpy as np

# 파일 경로
input_path = r"C:\Users\henaj\Downloads\파이널 프로젝트\기상데이터\일별_기상데이터.csv"
output_path = r"C:\Users\henaj\Downloads\파이널 프로젝트\기상데이터\일별_기상데이터_이상치 처리.csv"

# 데이터 불러오기
df = pd.read_csv(input_path)

# 대상 열
target_cols = ['평균 기온(°C)', '전년 기온(°C)']

# 이상치 탐색 및 처리
for col in target_cols:
    # 이상치 조건 (-50 이하인 값)
    outliers = df[df[col] <= -50]

    for idx, value in outliers[col].items():
        # 전날, 다음날 값 가져오기
        prev_val = df[col][idx - 1] if idx - 1 >= 0 else np.nan
        next_val = df[col][idx + 1] if idx + 1 < len(df) else np.nan

        # 앞뒤 평균 구하기 (둘 중 하나가 NaN이면 다른 하나 값으로 대체)
        if pd.notna(prev_val) and pd.notna(next_val):
            new_val = (prev_val + next_val) / 2
        elif pd.notna(prev_val):
            new_val = prev_val
        elif pd.notna(next_val):
            new_val = next_val
        else:
            new_val = np.nan  # 둘 다 없으면 NaN 처리

        # 변경 적용
        df.at[idx, col] = new_val

        # 변경 사항 출력
        print(f"({idx},{col}) {value} -> {new_val} 로 대체")

print("대체 완료!")

# 새로운 CSV 파일로 저장
df.to_csv(output_path, index=False, encoding='utf-8-sig')


(3108,평균 기온(°C)) -99.0 -> 3.3605633802816897 로 대체
(3514,평균 기온(°C)) -99.0 -> -1.036619718309859 로 대체
(3474,전년 기온(°C)) -99.0 -> 3.3605633802816897 로 대체
대체 완료!


In [2]:
import pandas as pd

# 파일 경로
input_path = r"C:\Users\henaj\Downloads\파이널 프로젝트\기상데이터\일별_기상데이터_이상치 처리.csv"
output_path = r"C:\Users\henaj\Downloads\파이널 프로젝트\기상데이터\일별_기상데이터_주간평균.csv"

# 데이터 불러오기
df = pd.read_csv(input_path)

# 조회일자를 datetime으로 변환
df['조회일자'] = pd.to_datetime(df['조회일자'], format='%Y%m%d')

# '평년 일사량(MJ/㎡)’, ‘평년 적설량(cm)’ 열 삭제 (있을 때만)   
if '평년 일사량(MJ/㎡)' in df.columns:
    df = df.drop(columns=['평년 일사량(MJ/㎡)'])

if '평년 적설량(cm)' in df.columns:
    df = df.drop(columns=['평년 적설량(cm)'])



# 7일 단위 그룹핑 (0부터 시작하는 주차 번호 생성)
df['week_group'] = (df.index // 7)

# 그룹별로 평균 계산
weekly_df = df.groupby('week_group').mean(numeric_only=True)

# 각 그룹의 시작일과 종료일 가져오기
group_ranges = df.groupby('week_group')['조회일자'].agg(['min', 'max'])

# 새로운 조회일자 형식으로 변환
weekly_df['조회일자'] = group_ranges['min'].dt.strftime('%Y%m%d') + '-' + group_ranges['max'].dt.strftime('%Y%m%d')

# 열 순서 조정 (조회일자가 맨 앞으로 오게)
cols = ['조회일자'] + [col for col in weekly_df.columns if col != '조회일자']
weekly_df = weekly_df[cols]

# 결과 저장
weekly_df.to_csv(output_path, index=False, encoding='utf-8-sig')

print("주간 평균 데이터 생성 완료!")



주간 평균 데이터 생성 완료!


In [3]:
# -*- coding: utf-8 -*-
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from scipy import stats
import math

# ---------- 한글 폰트 설정 추가 ----------
# 이 부분이 가장 중요합니다.
# matplotlib에 시스템의 한글 폰트를 설정하여 글씨 깨짐을 방지합니다.

from matplotlib import font_manager, rc

# 폰트 경로를 찾아서 설정합니다.
# Windows 환경의 경우
font_path = 'C:/Windows/Fonts/malgun.ttf'
# 만약 'malgun.ttf' 폰트가 없다면, 다른 한글 폰트(ex: NanumGothic.ttf) 경로를 지정해주세요.
if os.path.exists(font_path):
    font_name = font_manager.FontProperties(fname=font_path).get_name()
    rc('font', family=font_name)
else:
    print("맑은 고딕 폰트를 찾을 수 없습니다. 다른 한글 폰트를 사용하거나 설치해주세요.")
    # 기본 fallback 폰트로 설정하거나 다른 폰트 경로를 지정
    rc('font', family='AppleGothic') # macOS 환경
    # rc('font', family='NanumGothic') # 리눅스 환경 등

# 마이너스 부호가 깨지는 현상을 방지합니다.
plt.rcParams['axes.unicode_minus'] = False
# ---------------------------------------------


# ---------- 설정 ----------
base_path = r'C:\Users\henaj\Downloads\파이널 프로젝트\기상데이터'
input_file = os.path.join(base_path, '일별_기상데이터_주간평균.csv')
output_excel = os.path.join(base_path, '일별_기상데이터_주간평균_EDA.xlsx')
plots_folder = os.path.join(base_path, 'EDA_주간평균_plots')

os.makedirs(plots_folder, exist_ok=True)

# 분석할 열 목록 (질문에서 주신 열 그대로)
cols = [
    '평균 기온(°C)', '최고 기온(°C)', '최저 기온(°C)', '전년 기온(°C)', '평년 기온(°C)', '평년 최고 기온(°C)', '평년 최저 기온(°C)',
    '평균 강수량(mm)', '전년 강수량(mm)', '평년 강수량(mm)',
    '평균 일조시간(hr)', '전년 일조시간(hr)', '평년 일조시간(hr)',
    '평균 일사량(MJ/㎡)', '전년 일사량(MJ/㎡)', 
    '평균 습도(%)', '전년 습도(%)', '평년 습도(%)',
    '평균 운량(1/10)', '전년 운량(1/10)', '평년 운량(1/10)',
    '평균 적설량(cm)', '전년 적설량(cm)', 
    '평균 순간최대풍속(m/s)', '전년 순간최대풍속(m/s)', '평년 순간최대풍속(m/s)'
]

# ---------- 데이터 불러오기 ----------
# 인코딩 문제 발생 시 encoding='cp949'로 변경하세요.
df = pd.read_csv(input_file, encoding='utf-8-sig')

# '조회일자' 기준 정렬(필요시)
#if '조회일자' in df.columns:
    # 조회일자 숫자형(예: 20250630) -> 문자열로 변환 후 날짜 타입으로 바꾸면 편함
    #try:
        #df['조회일자_dt'] = pd.to_datetime(df['조회일자'].astype(str), format='%Y%m%d')
        #df = df.sort_values('조회일자_dt', ascending=False).drop(columns=['조회일자_dt'])
    #except Exception:
        # 변환 실패하면 그냥 넘어감
        #pass

# 필요한 열만 남기되, 존재하지 않는 열은 제외
available_cols = [c for c in cols if c in df.columns]
if len(available_cols) < 1:
    raise ValueError("분석할 열이 데이터에 없습니다. 파일의 열 이름을 확인해 주세요.")

df_sub = df[ ['조회일자'] + available_cols ].copy()

# 숫자형으로 변환 (문자열 중 쉼표, 공백 등 제거)
for c in available_cols:
    # 쉼표 제거, 공백 제거 후 numeric 변환
    df_sub[c] = pd.to_numeric(df_sub[c].astype(str).str.replace(',', '').str.strip(), errors='coerce')

# ---------- 요약 통계 ----------
summary = df_sub[available_cols].agg(['count','mean','median','min','max','std']).transpose()
summary = summary.rename(columns={'count':'N','median':'median','min':'min','max':'max','std':'std'})

# 결측치 정보
summary['missing_count'] = df_sub.shape[0] - summary['N']

# ---------- 정규성 검사 ----------
# 샘플 수가 5000 이하이면 Shapiro 권장, 그보다 크면 normaltest(D'Agostino) 사용
normality_results = []
n_rows = df_sub.shape[0]
for c in available_cols:
    series = df_sub[c].dropna()
    n = len(series)
    if n < 3:
        normality_results.append({'variable':c, 'test':'none', 'stat':np.nan, 'pvalue':np.nan, 'note':'not enough data'})
        continue
    try:
        if n <= 5000:
            stat, p = stats.shapiro(series)
            normality_results.append({'variable':c, 'test':'shapiro', 'stat':stat, 'pvalue':p,
                                     'note':('normal' if p>0.05 else 'not_normal')})
        else:
            # normaltest requires n>=8
            stat, p = stats.normaltest(series)
            normality_results.append({'variable':c, 'test':'normaltest', 'stat':stat, 'pvalue':p,
                                     'note':('normal' if p>0.05 else 'not_normal')})
    except Exception as e:
        normality_results.append({'variable':c, 'test':'error', 'stat':np.nan, 'pvalue':np.nan, 'note':str(e)})

normality_df = pd.DataFrame(normality_results).set_index('variable')

# ---------- 이상치 탐지 (IQR 방법) ----------
outlier_info = []
iqr_details = {}
for c in available_cols:
    s = df_sub[c].dropna()
    if len(s) == 0:
        outlier_info.append({'variable':c, 'n_outliers':0, 'pct_outliers':0.0, 'lower_bound':np.nan, 'upper_bound':np.nan})
        iqr_details[c] = {'Q1':np.nan,'Q3':np.nan,'IQR':np.nan}
        continue
    Q1 = s.quantile(0.25)
    Q3 = s.quantile(0.75)
    IQR = Q3 - Q1
    lower = Q1 - 1.5 * IQR
    upper = Q3 + 1.5 * IQR
    outliers = s[(s < lower) | (s > upper)]
    n_out = outliers.shape[0]
    pct = n_out / df_sub.shape[0] * 100
    outlier_info.append({'variable':c, 'n_outliers':n_out, 'pct_outliers':pct, 'lower_bound':lower, 'upper_bound':upper})
    iqr_details[c] = {'Q1':Q1,'Q3':Q3,'IQR':IQR}

outliers_df = pd.DataFrame(outlier_info).set_index('variable')

# ---------- 상관관계 (Pearson) 및 p-values ----------
vars_for_corr = available_cols.copy()
corr_matrix = df_sub[vars_for_corr].corr(method='pearson')

# p-value matrix
pval_matrix = pd.DataFrame(index=vars_for_corr, columns=vars_for_corr, dtype=float)
for i in range(len(vars_for_corr)):
    for j in range(len(vars_for_corr)):
        x = df_sub[vars_for_corr[i]].dropna()
        y = df_sub[vars_for_corr[j]].dropna()
        # To compute pearsonr, need aligned data: drop NaN pairs
        common = df_sub[[vars_for_corr[i], vars_for_corr[j]]].dropna()
        if common.shape[0] < 3:
            pval_matrix.iloc[i,j] = np.nan
        else:
            try:
                r, p = stats.pearsonr(common[vars_for_corr[i]], common[vars_for_corr[j]])
                pval_matrix.iloc[i,j] = p
            except Exception:
                pval_matrix.iloc[i,j] = np.nan

# ---------- 시각화 생성 ----------
# 1) 각 변수별 히스토그램 + 밀도 - 한글 폰트 적용
hist_files = []
for c in available_cols:
    fig, ax = plt.subplots(figsize=(8,5))
    series = df_sub[c].dropna()
    if len(series) > 0:
        ax.hist(series, bins=30, density=True, alpha=0.6, color='skyblue', edgecolor='black')
        try:
            density = stats.gaussian_kde(series)
            xs = np.linspace(series.min(), series.max(), 200)
            ax.plot(xs, density(xs), color='red', linewidth=2)
        except Exception:
            pass
    ax.set_title(f'Histogram & Density: {c}', fontsize=12, pad=15)
    ax.set_xlabel(c, fontsize=10)
    ax.set_ylabel('Density', fontsize=10)
    ax.grid(True, alpha=0.3)
    
    fname = os.path.join(plots_folder, f'hist_{c.replace("/", "_").replace(" ", "_").replace("(", "").replace(")", "").replace("°", "deg").replace("㎡", "m2")}.png')
    fig.tight_layout()
    fig.savefig(fname, dpi=150, bbox_inches='tight')
    plt.close(fig)
    hist_files.append(fname)

# 2) 박스플롯: 여러 변수 한 장에 (그리드)
fig, ax = plt.subplots(figsize=(max(8, len(available_cols)*0.25), 
                                max(6, len(available_cols)*0.25)))
df_sub[available_cols].boxplot(ax=ax, rot=90) # rot=90 → x축 레이블 회전
ax.set_title('Boxplots of Selected Variables')
fig.tight_layout()

boxplot_file = os.path.join(plots_folder, 'boxplots_all_vars.png')
fig.savefig(boxplot_file, dpi=150)
plt.close(fig)

# 3) 상관행렬 히트맵 (matplotlib만 사용)
# 간단한 heatmap을 직접 그림
fig, ax = plt.subplots(figsize=(max(8, len(available_cols)*0.25), max(6, len(available_cols)*0.25)))
cax = ax.imshow(corr_matrix.values, interpolation='nearest', vmin=-1, vmax=1)
ax.set_xticks(np.arange(len(vars_for_corr)))
ax.set_yticks(np.arange(len(vars_for_corr)))
ax.set_xticklabels(vars_for_corr, rotation=90, fontsize=8)
ax.set_yticklabels(vars_for_corr, fontsize=8)
fig.colorbar(cax, ax=ax, fraction=0.03)
ax.set_title('Pearson Correlation Matrix')
fig.tight_layout()
corr_file = os.path.join(plots_folder, 'correlation_matrix.png')
fig.savefig(corr_file, dpi=150)
plt.close(fig)

# ---------- 엑셀로 저장 (숫자 결과 + 이미지 삽입) ----------
with pd.ExcelWriter(output_excel, engine='xlsxwriter') as writer:
    # 요약 통계
    summary.to_excel(writer, sheet_name='summary_stats')
    # 정규성
    normality_df.to_excel(writer, sheet_name='normality_tests')
    # 이상치
    outliers_df.to_excel(writer, sheet_name='outlier_summary')
    # 상관계수 및 p-value
    corr_matrix.to_excel(writer, sheet_name='correlation_matrix')
    pval_matrix.to_excel(writer, sheet_name='correlation_pvalues')
    # 원본(선택한 열)도 저장
    df_sub.to_excel(writer, sheet_name='data_sample', index=False)

    # 이미지 삽입: 새로운 시트 생성
    workbook  = writer.book
    try:
        worksheet = workbook.add_worksheet('Plots')
    except Exception:
        worksheet = writer.sheets.get('Plots') or workbook.add_worksheet('Plots')

    # 이미지 위치 지정 (행, 열)
    row = 0
    col = 0
    # 먼저 히스토그램들 (한 행에 2개씩)
    for i, fpath in enumerate(hist_files):
        # Excel cell 위치 (row, col) in pixels offset handled by xlsxwriter
        worksheet.insert_image(row, col, fpath, {'x_scale':0.9, 'y_scale':0.9})
        col += 8  # 다음 이미지 위한 열 이동
        if (i+1) % 2 == 0:
            row += 20
            col = 0
    # 박스플롯
    row += 22
    col = 0
    worksheet.insert_image(row, col, boxplot_file, {'x_scale':0.9, 'y_scale':0.9})
    row += 30
    # 상관행렬 이미지
    worksheet.insert_image(row, 0, corr_file, {'x_scale':0.9, 'y_scale':0.9})

# ---------- 콘솔 출력 요약 ----------
print("EDA 완료.")
print(f"결과 엑셀 파일: {output_excel}")
print(f"플롯 이미지 폴더: {plots_folder}")
print("\n요약 통계 (모든 변수):")
print(summary)
print("\n정규성 테스트 결과:")
print(normality_df)
print("\n이상치 요약 (모든 변수):")
print(outliers_df.sort_values('n_outliers', ascending=False))
print("\n상관계수:")
print(corr_matrix)

# 끝

EDA 완료.
결과 엑셀 파일: C:\Users\henaj\Downloads\파이널 프로젝트\기상데이터\일별_기상데이터_주간평균_EDA.xlsx
플롯 이미지 폴더: C:\Users\henaj\Downloads\파이널 프로젝트\기상데이터\EDA_주간평균_plots

요약 통계 (모든 변수):
                    N       mean     median        min        max        std  \
평균 기온(°C)       522.0  13.459362  14.507425  -7.148529  29.510504   9.224417   
최고 기온(°C)       522.0  18.851459  20.480462  -2.423950  35.200840   9.072099   
최저 기온(°C)       522.0   8.854430   8.911713 -11.844989  25.492354   9.669864   
전년 기온(°C)       522.0  13.290803  13.812395  -6.444350  29.500840   9.052659   
평년 기온(°C)       522.0  12.093951  12.773667  -1.172708  25.217697   8.597635   
평년 최고 기온(°C)    522.0  17.337426  18.754070   3.832983  29.646482   8.372922   
평년 최저 기온(°C)    522.0   7.581882   7.279104  -5.608529  21.883501   8.977884   
평균 강수량(mm)      522.0   3.593123   1.576047   0.000000  34.896177   5.085037   
전년 강수량(mm)      522.0   3.562007   1.821271   0.000000  29.985513   4.748664   
평년 강수량(mm)      522.0   3.520580   2.

In [1]:
# 경로가 'C:\Users\henaj\Downloads\파이널 프로젝트\기상데이터'인 '일별_기상데이터_주간평균_날씨파생변수.xlsx' 
#파일에서 '조회일자' 열 값이 '20151020-20151026'인 열부터만 남겨두고 그 위의 열은 삭제하기(단, 첫번째 행에 있는
#열 이름은 제외하고 삭제하기). 그렇게 해서 '기상데이터_주간평균_날씨파생변수_열삭제.csv'라는 csv 형식의 파일로
#같은 경로에 저장해두기.

import pandas as pd
import os

# 파일 경로 설정
base_path = r'C:\Users\henaj\Downloads\파이널 프로젝트\기상데이터'
input_file = os.path.join(base_path, '일별_기상데이터_주간평균_날씨파생변수.xlsx')
output_file = os.path.join(base_path, '기상데이터_주간평균_날씨파생변수_열삭제.csv')

# 엑셀 파일 불러오기
df = pd.read_excel(input_file)

# '조회일자' 기준 인덱스 찾기
start_index = df[df['조회일자'] == '20151020-20151026'].index[0]

# 해당 인덱스부터 데이터 남기기 (열 이름은 유지됨)
df_filtered = df.loc[start_index:].reset_index(drop=True)

# CSV로 저장
df_filtered.to_csv(output_file, index=False, encoding='utf-8-sig')

print("완료! 필터링된 데이터가 저장되었습니다:", output_file)



완료! 필터링된 데이터가 저장되었습니다: C:\Users\henaj\Downloads\파이널 프로젝트\기상데이터\기상데이터_주간평균_날씨파생변수_열삭제.csv
